# Training with gpu

In [ ]:
from __future__ import print_function
from sklearn.model_selection import StratifiedKFold as SKFold
import os
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu1,floatX=float32"
import eeg_cnn_funcs
from eeg_cnn_funcs import *
import numpy as np
import os
import random

In [ ]:
path_to_data = '/home/moskaleona/alenadir/data/dataSepWins/'
path_to_results = '/home/moskaleona/alenadir/results/pair_learning/'
file_aucs = '/home/moskaleona/alenadir/results/pair_learning/aucs.txt'
subnums = [25,26,27,28,29,30,32,33,34,35,36,37,38]

In [ ]:
elNum, locs_2d = loadlocs(path_to_data)

In [ ]:
learning_rate = 1e-4
l1_coef = 1e-5
l2_coef = 1e-3
dropout = 0.75
#hyperparams = [[5e-4, 1e-4, 1e-5, 0.25],
#               [1e-4, 1e-5, 1e-3, 0.75]]

## Pair learning
Concatenation of subject A and subject B and then trainig and validation on such data.

In [ ]:
from sklearn.model_selection import train_test_split
n_epochs = 2
for i_sub in subnums:
    i_X= scipy.io.loadmat(os.path.join(path_to_data,'FeatureMat_timeWin' + str(i_sub) +'.mat'))['features']
    i_y = np.squeeze(i_X[:, -1])
    for j_sub in subnums:
        if i_sub == j_sub:
            continue
        elif i_sub < j_sub:
            seed = 1
        else:
            seed = 2  
        path = os.path.join(path_to_results, str(i_sub)+str(j_sub)) + '/'
        if not os.path.isdir(path):
            os.mkdir(path)
        
        j_X = scipy.io.loadmat(os.path.join(path_to_data, 'FeatureMat_timeWin' + str(j_sub) +'.mat'))['features']
        j_y = np.squeeze(j_X[:, -1])
        
        i_X_train, i_X_test, i_y_train, i_y_test = train_test_split(i_X, i_y, test_size=0.2, random_state=seed)
        j_X_train, j_X_test, j_y_train, j_y_test = train_test_split(j_X, j_y, test_size=0.2, random_state=seed)
        
        X_train = np.vstack((i_X_train, j_X_train))
        y_train = np.hstack((i_y_train, j_y_train))
        X_test = np.vstack((i_X_test, j_X_test))
        y_test= np.hstack((i_y_test, j_y_test))
        
        np.random.seed(seed)
        train_ind = np.arange(X_train.shape[0])
        test_ind = np.arange(X_test.shape[0])
        np.random.shuffle(train_ind)
        np.random.shuffle(test_ind)
        X_train = X_train[train_ind]
        X_test = X_test[test_ind]
        y_train = y_train[train_ind]
        y_test = y_test[test_ind]
        
        print('Xtrain=%s, Xtest=%s, ytrain=%s, ytest=%s'%(X_train.shape,X_test.shape,y_train.shape, y_test.shape))
        print(y_train.mean(), y_test.mean())

        # Conv-LSTM Mode
        print('Generating images for all time windows...')
        images_train = np.array([gen_images(np.array(locs_2d), X_train[:, i * (3 * elNum):(i + 1) * (3 * elNum)],
                                          32, normalize=False) for i in
                                range(X_train.shape[1] / (3 * elNum))
                                ])
        images_test = np.array([gen_images(np.array(locs_2d), X_test[:, i * (3 * elNum):(i + 1) * (3 * elNum)],
                                          32, normalize=False) for i in
                                range(X_test.shape[1] / (3 * elNum))
                                ])

        print('\n')

    
        # Training (LSTM)
        print('Training the LSTM-CONV Model for {0}&{1} .'.format(i_sub, j_sub) +
                    'Params: lr={0}, l1={1}, l2={2}, dr={3}'.format(learning_rate, l1_coef, l2_coef, dropout))
    
        with open(file_aucs, 'a') as f:
            f.write('Auc for %d & %d: '%(i_sub, j_sub))
        train_test(images_train, images_test, y_train, y_test,
                            'mix', path, filename=file_aucs, num_epochs=n_epochs,
                            learning_rate=learning_rate, l1_coef=l1_coef, l2_coef=l2_coef, dropout=dropout)


In [ ]:
# Output was overloaded, so it finished in a wrong way in the middle of the cycle
from sklearn.model_selection import train_test_split
n_epochs = 200
for i_sub in [33,]: #[34, 35, 36, 37, 38]:
    i_X= scipy.io.loadmat(os.path.join(path_to_data,'FeatureMat_timeWin' + str(i_sub) +'.mat'))['features']
    i_y = np.squeeze(i_X[:, -1])
    for j_sub in [38,]:#subnums:
        if i_sub == j_sub:
            continue
        elif i_sub < j_sub:
            seed = 1
        else:
            seed = 2  
        path = os.path.join(path_to_results, str(i_sub)+str(j_sub)) + '/'
        if not os.path.isdir(path):
            os.mkdir(path)
        
        j_X = scipy.io.loadmat(os.path.join(path_to_data, 'FeatureMat_timeWin' + str(j_sub) +'.mat'))['features']
        j_y = np.squeeze(j_X[:, -1])
        
        i_X_train, i_X_test, i_y_train, i_y_test = train_test_split(i_X, i_y, test_size=0.2, random_state=seed)
        j_X_train, j_X_test, j_y_train, j_y_test = train_test_split(j_X, j_y, test_size=0.2, random_state=seed)
        
        X_train = np.vstack((i_X_train, j_X_train))
        y_train = np.hstack((i_y_train, j_y_train))
        X_test = np.vstack((i_X_test, j_X_test))
        y_test= np.hstack((i_y_test, j_y_test))
        
        np.random.seed(seed)
        train_ind = np.arange(X_train.shape[0])
        test_ind = np.arange(X_test.shape[0])
        np.random.shuffle(train_ind)
        np.random.shuffle(test_ind)
        X_train = X_train[train_ind]
        X_test = X_test[test_ind]
        y_train = y_train[train_ind]
        y_test = y_test[test_ind]

        # Conv-LSTM Mode
        # print('Generating images for all time windows...')
        images_train = np.array([gen_images(np.array(locs_2d), X_train[:, i * (3 * elNum):(i + 1) * (3 * elNum)],
                                          32, normalize=False) for i in
                                range(X_train.shape[1] / (3 * elNum))
                                ])
        images_test = np.array([gen_images(np.array(locs_2d), X_test[:, i * (3 * elNum):(i + 1) * (3 * elNum)],
                                          32, normalize=False) for i in
                                range(X_test.shape[1] / (3 * elNum))
                                ])

        print('\n')

    
        # Training (LSTM)
        print('Training the LSTM-CONV Model for {0}&{1} .'.format(i_sub, j_sub) +
                    'Params: lr={0}, l1={1}, l2={2}, dr={3}'.format(learning_rate, l1_coef, l2_coef, dropout))
    
        with open(file_aucs, 'a') as f:
            f.write('Auc for %d & %d: '%(i_sub, j_sub))
        train_test(images_train, images_test, y_train, y_test,
                            'mix', path, filename=file_aucs, num_epochs=n_epochs,
                            learning_rate=learning_rate, l1_coef=l1_coef, l2_coef=l2_coef, dropout=dropout)


## Training on A, testing on B

In [ ]:
file_aucs_AB = '/home/moskaleona/alenadir/results/pair_learning/aucsAB.txt'
n_epochs = 200
for i_sub in subnums:
    X_train = scipy.io.loadmat(os.path.join(path_to_data,'FeatureMat_timeWin' + str(i_sub) +'.mat'))['features']
    y_train = np.squeeze(X_train[:, -1])
    for j_sub in subnums:
        if i_sub == j_sub:
            continue
        elif i_sub < j_sub:
            seed = 1
        else:
            seed = 2  
        path = os.path.join(path_to_results, str(i_sub)+str(j_sub)) + '/'
        if not os.path.isdir(path):
            os.mkdir(path)
        
        X_test = scipy.io.loadmat(os.path.join(path_to_data, 'FeatureMat_timeWin' + str(j_sub) +'.mat'))['features']
        y_test = np.squeeze(X_test[:, -1])
        
        np.random.seed(seed)
        train_ind = np.arange(X_train.shape[0])
        test_ind = np.arange(X_test.shape[0])
        np.random.shuffle(train_ind)
        np.random.shuffle(test_ind)
        X_train = X_train[train_ind]
        X_test = X_test[test_ind]
        y_train = y_train[train_ind]
        y_test = y_test[test_ind]
        
        print('Xtrain=%s, Xtest=%s, ytrain=%s, ytest=%s'%(X_train.shape,X_test.shape,y_train.shape, y_test.shape))
        print(y_train.mean(), y_test.mean())

        # Conv-LSTM Mode
        print('Generating images for all time windows...')
        images_train = np.array([gen_images(np.array(locs_2d), X_train[:, i * (3 * elNum):(i + 1) * (3 * elNum)],
                                          32, normalize=False) for i in
                                range(X_train.shape[1] / (3 * elNum))
                                ])
        images_test = np.array([gen_images(np.array(locs_2d), X_test[:, i * (3 * elNum):(i + 1) * (3 * elNum)],
                                          32, normalize=False) for i in
                                range(X_test.shape[1] / (3 * elNum))
                                ])

        print('\n')

    
        # Training (LSTM)
        print('Training the LSTM-CONV Model on {0}, testing on {1} .'.format(i_sub, j_sub) +
                    'Params: lr={0}, l1={1}, l2={2}, dr={3}'.format(learning_rate, l1_coef, l2_coef, dropout))
    
        with open(file_aucs_AB, 'a') as f:
            f.write('Auc for train on %d, test on %d: '%(i_sub, j_sub))
        train_test(images_train, images_test, y_train, y_test,
                            'mix', path, filename=file_aucs_AB, num_epochs=n_epochs,
                            learning_rate=learning_rate, l1_coef=l1_coef, l2_coef=l2_coef, dropout=dropout)


## Training on subjects together

In [ ]:
# Data aggregation for training on all choosen subjects together
feats = scipy.io.loadmat(path_to_data + 'FeatureMat_timeWin32.mat')['features']
print(feats.shape)
feats = np.concatenate((feats, scipy.io.loadmat(path_to_data + 'FeatureMat_timeWin33.mat')['features']))
feats = np.concatenate((feats, scipy.io.loadmat(path_to_data + 'FeatureMat_timeWin34.mat')['features']))

labels = np.squeeze(feats[:, -1])
print(feats.shape)
print(feats[:,-1])

In [ ]:
# Training on subjects together
n_folds = 4
nepochs = 250
print('Generating images for all time windows...')
images_timewin = np.array([gen_images(np.array(locs_2d), feats[:, i * (3 * elNum):(i + 1) * (3 * elNum)],
                                        32, normalize=False) for i in
                            range(feats.shape[1] / (3 * elNum))
                            ])
print('\n')

    
fold_pairs = []
skf = SKFold(n_splits=n_folds, shuffle=True, random_state=1)
np.random.seed(108)
random.seed(108)
for tr, ts in skf.split(images_timewin[0,:], labels):
    random.shuffle(tr)
    random.shuffle(ts)
    print(len(tr), len(ts), np.sum(labels[tr]), np.sum(labels[ts]))
    fold_pairs.append((tr,ts))
    
print(len(labels), np.sum(labels))
# Training (LSTM)

for params in hyperparams:
    print('Training the LSTM-CONV Model ' +
            'Params: lr={0}, l1={1}, l2={2}, dr={3}'.format(params[0], params[1], params[2], params[3]))
    for fold in range(2):
        path = os.path.join(path_to_results, '323334', str(fold)) + '/'
        if not os.path.isdir(path):
            os.mkdir(path)
        print('fold:', fold)
        train_save(images_timewin, labels, fold_pairs[fold],
                        'mix', path, num_epochs=nepochs,
                        learning_rate=params[0], l1_coef=params[1], l2_coef=params[2], dropout=params[3], n_timewin=4)

## Training on different subjects

In [ ]:
%%time
# Training on different subjects
n_folds = 4
nepochs = 250
for subnum in subnums:
    if not os.path.isdir(path_to_results + str(subnum)):
        os.mkdir(path_to_results + str(subnum))
    feats = scipy.io.loadmat(path_to_data + 'FeatureMat_timeWin' + str(subnum) +'.mat')['features']
    labels = np.squeeze(feats[:, -1])
    # Conv-LSTM Mode
    print('Generating images for all time windows...')
    images_timewin = np.array([gen_images(np.array(locs_2d), feats[:, i * (3 * elNum):(i + 1) * (3 * elNum)],
                                          32, normalize=False) for i in
                                range(feats.shape[1] / (3 * elNum))
                                ])

    print('\n')

    
    fold_pairs = []
    np.random.seed(108)
    skf = SKFold(n_splits=n_folds, shuffle=True)
    for tr, ts in skf.split(images_timewin[0,:], labels):
        random.shuffle(tr)
        random.shuffle(ts)
        print(len(tr), len(ts), np.sum(labels[tr]), np.sum(labels[ts]))
        fold_pairs.append((tr,ts))
    
    print(len(labels), np.sum(labels))
    # Training (LSTM)
    if subnum == 33:
        for params in hyperparams33:
            print('Training the LSTM-CONV Model for {0} subject.'.format(subnum) +
                'Params: lr={0}, l1={1}, l2={2}, dr={3}'.format(params[0], params[1], params[2], params[3]))
            for fold in range(2):
                path = path_to_results + str(subnum) + '/' + str(fold) + '/'
                if not os.path.isdir(path):
                    os.mkdir(path)
                print('fold:', fold)
                train_save(images_timewin, labels, fold_pairs[fold],
                            'mix', path, num_epochs=nepochs,
                            learning_rate=params[0], l1_coef=params[1], l2_coef=params[2], dropout=params[3], n_timewin=4)
    else:
        for params in hyperparams:
            print('Training the LSTM-CONV Model for {0} subject.'.format(subnum) +
                    'Params: lr={0}, l1={1}, l2={2}, dr={3}'.format(params[0], params[1], params[2], params[3]))
            for fold in range(2):
                path = path_to_results + str(subnum) + '/' + str(fold) + '/'
                if not os.path.isdir(path):
                    os.mkdir(path)
                print('fold:', fold)
                train_save(images_timewin, labels, fold_pairs[fold],
                            'mix', path, num_epochs=nepochs,
                            learning_rate=params[0], l1_coef=params[1], l2_coef=params[2], dropout=params[3], n_timewin=4)


## Plotting images

In [ ]:
import matplotlib.pyplot as plt
import time

In [ ]:
%matplotlib inline
subject = np.random.randint(images_train.shape[1])
time=np.random.randint(4)
image = np.swapaxes(images_train, 2,4)[time, subject,:]
image = image / np.max(image) - np.min(image) / np.max(image)
plt.imshow(image)